In [12]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator
from tfx.proto import example_gen_pb2, trainer_pb2, transform_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [13]:
PIPELINE_NAME = 'churn-pipeline'
SCHEMA_PIPELINE_NAME = 'churn-tfdv-schema'

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = 'data'

In [14]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [15]:
input = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train*'),
    example_gen_pb2.Input.Split(name='val', pattern='val*')
])
example_gen = CsvExampleGen(input_base=DATA_ROOT, input_config=input)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [17]:
context.show(statistics_gen.outputs['statistics'])

In [18]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [19]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'Churn',INT,required,,-
'Contract Length',STRING,required,,'Contract Length'
'CustomerID',INT,required,,-
'Gender',STRING,required,,'Gender'
'Last Interaction',INT,required,,-
'Payment Delay',INT,required,,-
'Subscription Type',STRING,required,,'Subscription Type'
'Support Calls',INT,required,,-


,Values
Domain,
'Contract Length',"'Annual', 'Monthly', 'Quarterly'"
'Gender',"'Female', 'Male'"
'Subscription Type',"'Basic', 'Premium', 'Standard'"


In [20]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [21]:
context.show(example_validator.outputs['anomalies'])

In [22]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(os.path.join('modules', 'transform.py')),
)

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmp730yflyk
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmp730yflyk
copying build/lib/transform.py -> /tmp/tmp730yflyk
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info


/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp730yflyk/tfx_user_code_Transform-0.0+95aee4eddfd01ec9e160a6a297feffb7171a9f2d1fa7f408e72a10a20678c380-py3.10.egg-info
running install_scripts
creating /tmp/tmp730yflyk/tfx_user_code_Transform-0.0+95aee4eddfd01ec9e160a6a297feffb7171a9f2d1fa7f408e72a10a20678c380.dist-info/WHEEL
creating '/tmp/tmpjnmun1kn/tfx_user_code_Transform-0.0+95aee4eddfd01ec9e160a6a297feffb7171a9f2d1fa7f408e72a10a20678c380-py3-none-any.whl' and adding '/tmp/tmp730yflyk' to it
adding 'trainer.py'
adding 'transform.py'
adding 'tfx_user_code_Transform-0.0+95aee4


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Transform-0.0+95aee4eddfd01ec9e160a6a297feffb7171a9f2d1fa7f408e72a10a20678c380-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [29]:
trainer = Trainer(
    module_file=os.path.abspath(os.path.join('modules', 'trainer.py')),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['val'], num_steps=200)
)

context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib


/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmpw4so8tug
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmpw4so8tug
copying build/lib/transform.py -> /tmp/tmpw4so8tug
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpw4so8tug/tfx_user_code_Trainer-0.0+83ccbf5039cd570b1c59fb197589cf876e039177eeb1efa25ca98ad0e4e8fd43-py3.10.egg-info
running install_scripts
creating /tmp/tmpw4so8tug/tfx_user_code_Trainer-0.0+83ccbf5039cd570b1c59fb197589cf876e039177eeb1efa25ca98ad0e4e8fd43.dist-info/WHEEL
creating '/tmp/tmp89uboiq5/

Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Trainer-0.0+83ccbf5039cd570b1c59fb197589cf876e039177eeb1efa25ca98ad0e4e8fd43-py3-none-any.whl



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/12/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/12/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)